In [11]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.impute import KNNImputer
from imblearn.over_sampling import SMOTE

# Load dataset
df = pd.read_csv('f1_2019_to_2022_all_drivers_all_data.csv', low_memory=False)

# Convert 'LapTime' and sector times to seconds
time_columns = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']
for col in time_columns:
    df[col] = pd.to_timedelta(df[col]).dt.total_seconds()

# Keep the original 'Driver' and 'Circuit' for EDA, Preprocessing before one hot encoding
df['Original_Driver'] = df['Driver']
df['Original_Circuit'] = df['Circuit']

# Encode Rainfall, FreshTyre, IsAccurate, and IsPersonalBest to integer
df['Rainfall'] = df['Rainfall'].astype(int)
df['FreshTyre'] = df['FreshTyre'].astype(int)
df['IsAccurate'] = df['IsAccurate'].astype(int)

# Drop columns not related to lap time prediction
columns_to_drop = ['Time', 'LapStartTime', 'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
                   'PitOutTime', 'PitInTime', 'LapStartDate', 'Deleted', 'DeletedReason', 'FastF1Generated','IsPersonalBest', 'Sector3Time','Sector2Time','Sector1Time']
df.drop(columns=columns_to_drop, inplace=True) # Sector 3 times are deleted since it could shadow impact of weather features

# One-hot encoding
df = pd.get_dummies(df, columns=['Driver', 'Circuit', 'Compound', 'Team'])

# Feature Engineering: Categorize weather and return numerical labels for models Initial: 0 , 25 , 19 
def categorize_weather(row):
    if row['Rainfall'] > 0:
        return 'Rainy'
    elif row['AirTemp'] > 27:
        return 'Hot'
    elif row['AirTemp'] > 20:
        return 'Warm'
    else:
        return 'Cool'
df['Weather_Category'] = df.apply(categorize_weather, axis=1)

# Keep for further steps 
df['Original_Weather_Category'] = df['Weather_Category']
# Apply one-hot encoding to the 'Weather_Category' column
df = pd.get_dummies(df, columns=['Weather_Category'])

# Create Track temperature category based on the result of clustering 
df['TrackTemp_Cat'] = pd.cut(df['TrackTemp'], bins=[18, 27, 34, 41, 50, np.inf], labels=['VERY_LOW', 'Low', 'Medium', 'High', 'VERY_HIGH'])
df['Original_TrackTemp_Cat'] = df['TrackTemp_Cat']
# One hot encoding for TrackTemp_cat
df = pd.get_dummies(df, columns=['TrackTemp_Cat'])

# Tyre Age Interaction with TrackTemp 
df['TyreAge_TrackTemp'] = df['TyreLife'] * df['TrackTemp']

# Standardize numeric features
numeric_features = ['SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'AirTemp', 'Humidity', 'Pressure', 'TrackTemp', 'WindDirection', 'WindSpeed','TrackStatus','Position','LapTime']
scaler = StandardScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])

# Impute missing values
imputer = KNNImputer(n_neighbors=5)
df[numeric_features] = imputer.fit_transform(df[numeric_features])

# Drop original categorical columns which were used for EDA
df = df.drop(['Original_Driver', 'Original_Circuit','Original_Weather_Category','Original_TrackTemp_Cat'], axis= 1)
# 1. Separate LapTime as dry or wet(rainy) condition ( since lapTime of rainy day would be recognized as outliers)
# 2. Remove Outliers for dry condition LapTime
# 3. Build Combined LapTime df (Outliers for dry days are deleted)

# Flag for rainy conditions
df['IsRainy'] = df['Rainfall'].apply(lambda x: 1 if x > 0 else 0)

df_dry = df[df['IsRainy'] == 0]
#df for rainy days 
df_wet = df[df['IsRainy'] == 1]


# Remove Outliers for dry days
Q1_dry = df_dry['LapTime'].quantile(0.25)
Q3_dry = df_dry['LapTime'].quantile(0.75)
IQR_dry = Q3_dry - Q1_dry
lower_bound_dry = Q1_dry - 1.5 * IQR_dry
upper_bound_dry = Q3_dry + 1.5 * IQR_dry
#Only dry days (Outliers Removed)
df_dry_filtered = df_dry[(df_dry['LapTime'] >= lower_bound_dry) & (df_dry['LapTime'] <= upper_bound_dry)]



# Build combined df for both dry-rainy days (Outliers are removed with rainy days lapTimes) 
df_combined = pd.concat([df_dry_filtered, df_wet], ignore_index=True)


df_combined.info(verbose=True)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79375 entries, 0 to 79374
Data columns (total 116 columns):
 #    Column                               Dtype  
---   ------                               -----  
 0    DriverNumber                         int64  
 1    LapTime                              float64
 2    LapNumber                            float64
 3    Stint                                float64
 4    SpeedI1                              float64
 5    SpeedI2                              float64
 6    SpeedFL                              float64
 7    SpeedST                              float64
 8    TyreLife                             float64
 9    FreshTyre                            int64  
 10   TrackStatus                          float64
 11   Position                             float64
 12   IsAccurate                           int64  
 13   AirTemp                              float64
 14   Humidity                             float64
 15   Pressure         

In [12]:
nan_counts = df_wet.isna().sum()
# Print columns with NaN count more than 0
print(nan_counts[nan_counts >0])

LapTime    181
dtype: int64


In [13]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X = df_combined.drop(['LapTime'], axis=1)
y = df_combined['LapTime']

# Split Train and test set for a model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Fitting a RandomForest model
model = RandomForestRegressor(n_estimators=100,max_depth=20,min_samples_split=5,random_state=42)
# Best Parameters: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 300}

model.fit(X_train, y_train)

# Print RMSE value of the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = mse ** 0.5
print(f"RMSE: {rmse}")

# Shows Important features
importance = model.feature_importances_

# Sort the feature importance
sorted_indices = np.argsort(importance)[::-1]

for index in sorted_indices:
    print(f"{X_train.columns[index]}: {importance[index]}")

ValueError: Input y contains NaN.

In [ ]:


features = X_train.columns

# List of weather-related feature patterns
weather_features = [
    'Pressure', 'AirTemp', 'Humidity', 'TrackTemp',
    'WindDirection', 'WindSpeed', 'Rainfall',
    'Weather_Category_','Rainfall','TrackTemp_Cat_VERY_LOW','TrackTemp_Cat_Low','Weather_Category_Warm','TrackTemp_Cat_High','TrackTemp_Cat_VERY_HIGH'
]

# Calculate the total importance of weather-related features
total_weather_importance = sum(
    importance for feature, importance in zip(features, importance)
    if any(weather_feature in feature for weather_feature in weather_features)
)

print(f"Total importance of weather-related features: {total_weather_importance}")

# Now let's calculate the importance for the circuit feature as a whole
circuit_importance_sum = sum(
    importance for feature, importance in zip(features, importance)
    if 'Circuit_' in feature
)

print(f"Total importance of circuit features: {circuit_importance_sum}")
